# Part - 1

### Generate dataset

In [1]:
import numpy as np
import sys

In [2]:
num_examples = 1000
output_dim = 12
#iterations = 1000

In [3]:
def int2vec(x,dim=output_dim):
    
    out = np.zeros(dim) # set output vector = dim number of 0s

    binrep = np.array(list(np.binary_repr(x))).astype('int') # get binary representation of the integer

    out[-len(binrep):] = binrep # set the last k values to binrep

    return out

In [4]:
def generate_dataset(output_dim = 8,num_examples=1000):
    
    # np.random.rand(num_examples) = randomly pick num_examples number of real numbers between (0,1)
    # multiple each by pow(2,(output_dim - 1)) and get their integer part

    x_left_int = (np.random.rand(num_examples) * 2**(output_dim - 1)).astype('int')
    x_right_int = (np.random.rand(num_examples) * 2**(output_dim - 1)).astype('int')
    y_int = x_left_int + x_right_int
    
    # Create 2 set of numbers and third set of numbers be their sum 
    
    
    x = list()
    for i in range(len(x_left_int)):
        x.append(np.concatenate((int2vec(x_left_int[i]),int2vec(x_right_int[i]))))
        #just concatenate the binary of 2 integers

    y = list()
    for i in range(len(y_int)):
        y.append(int2vec(y_int[i]))
        #get binary of sum

    x = np.array(x) # of length = 2 * output_dim
    y = np.array(y) # of length = output_dim
    
    return (x,y)

In [5]:
x,y = generate_dataset(num_examples=num_examples, output_dim = output_dim)
print("Input: two concatenated binary values:")
print(x[0])
print len(x[0])
print("\nOutput: binary value of their sum:")
print(y[0])
print len(y[0])

print "\n\n", len(x), len(y)

Input: two concatenated binary values:
[ 0.  0.  1.  0.  0.  0.  0.  1.  1.  0.  0.  1.  0.  0.  0.  1.  0.  1.
  0.  1.  1.  0.  0.  0.]
24

Output: binary value of their sum:
[ 0.  0.  1.  1.  0.  1.  1.  1.  0.  0.  0.  1.]
12


1000 1000


# Network

In [6]:
np.random.seed(1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [8]:
batch_size = 10
alpha = 0.1
iterations = 500

#define varous layer sizes
input_dim = len(x[0])   #24
layer_1_dim = 128
layer_2_dim = 64
output_dim = len(y[0])  #12

#initialise weights
weights_0_1 = (np.random.randn(input_dim,layer_1_dim) * 0.2) - 0.1
weights_1_2 = (np.random.randn(layer_1_dim,layer_2_dim) * 0.2) - 0.1
weights_2_3 = (np.random.randn(layer_2_dim,output_dim) * 0.2) - 0.1


for iter in range(iterations):
    error = 0

    for batch_i in range(int(len(x) / batch_size)):
        batch_x = x[(batch_i * batch_size):(batch_i+1)*batch_size]
        batch_y = y[(batch_i * batch_size):(batch_i+1)*batch_size]    

        layer_0 = batch_x
        layer_1 = sigmoid(layer_0.dot(weights_0_1))
        layer_2 = sigmoid(layer_1.dot(weights_1_2))
        layer_3 = sigmoid(layer_2.dot(weights_2_3))

        layer_3_delta = (layer_3 - batch_y) * layer_3  * (1 - layer_3)
        layer_2_delta = layer_3_delta.dot(weights_2_3.T) * layer_2 * (1 - layer_2)
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * layer_1 * (1 - layer_1)

        weights_0_1 -= layer_0.T.dot(layer_1_delta) * alpha
        weights_1_2 -= layer_1.T.dot(layer_2_delta) * alpha
        weights_2_3 -= layer_2.T.dot(layer_3_delta) * alpha

        error += (np.sum(np.abs(layer_3_delta)))

    sys.stdout.write("\rIter:" + str(iter) + " Loss:" + str(error))
    if(iter % 100 == 99):
        print("")

Iter:99 Loss:801.912923083
Iter:199 Loss:119.33355841
Iter:299 Loss:17.3692312253
Iter:399 Loss:7.99254301716
Iter:499 Loss:4.9352151436


# OOPs way

In [9]:
class Layer(object):
    
    def __init__(self,input_dim, output_dim,nonlin,nonlin_deriv):
        
        self.weights = (np.random.randn(input_dim, output_dim) * 0.2) - 0.1
        self.nonlin = nonlin
        self.nonlin_deriv = nonlin_deriv
    
    def forward(self,input):
        self.input = input
        self.output = self.nonlin(self.input.dot(self.weights))
        return self.output
    
    def backward(self,output_delta):
        self.weight_output_delta = output_delta * self.nonlin_deriv(self.output)
        return self.weight_output_delta.dot(self.weights.T)
    
    def update(self,alpha=0.1):
        self.weights -= self.input.T.dot(self.weight_output_delta) * alpha



In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_out2deriv(out):
    return out * (1 - out)

In [ ]:
layer_1 = Layer(input_dim,layer_1_dim,sigmoid,sigmoid_out2deriv)
layer_2 = Layer(layer_1_dim,layer_2_dim,sigmoid,sigmoid_out2deriv)
layer_3 = Layer(layer_2_dim, output_dim,sigmoid, sigmoid_out2deriv)

for iter in range(iterations):
    error = 0

    for batch_i in range(int(len(x) / batch_size)):
        batch_x = x[(batch_i * batch_size):(batch_i+1)*batch_size]
        batch_y = y[(batch_i * batch_size):(batch_i+1)*batch_size]  
        
        layer_1_out = layer_1.forward(batch_x)
        layer_2_out = layer_2.forward(layer_1_out)
        layer_3_out = layer_3.forward(layer_2_out)

        layer_3_delta = layer_3_out - batch_y
        layer_2_delta = layer_3.backward(layer_3_delta)
        layer_1_delta = layer_2.backward(layer_2_delta)
        layer_1.backward(layer_1_delta)
        
        layer_1.update()
        layer_2.update()
        layer_3.update()
        
        error += (np.sum(np.abs(layer_3_delta)))

    sys.stdout.write("\rIter:" + str(iter) + " Loss:" + str(error))
    if(iter % 100 == 99):
        print("")


Iter:99 Loss:3662.15339036
Iter:199 Loss:745.604224886
Iter:299 Loss:300.484057628
Iter:399 Loss:214.779353153
Iter:409 Loss:209.551468469